In [1]:
import pickle
import glob
import os
import pandas as pd
import numpy as np
import heapq

In [2]:
test_news_path = '../infer/mind/large_test/news.tsv'
bias_detection_path = 'data/0929_concat_mind_over_res.csv'
infer_dir = '../infer/interest_infer_naml/'
infer_dir_processed = '../infer/interest_infer_naml/processed'
candidate_news_file = 'data/news_new_with_time.tsv'

In [4]:
!ls -ahl $infer_dir_processed

total 132904
drwxr-xr-x  14 qin  staff   448B  8 May 02:49 .
drwxr-xr-x  14 qin  staff   448B  8 May 02:49 ..
-rw-r--r--   1 qin  staff    34M 17 May 05:57 candiate_news_vec.npy
-rw-r--r--   1 qin  staff   351K 17 May 05:57 candiate_nid_list.pkl
-rw-r--r--   1 qin  staff   3.1M 17 May 05:57 interest_behaviors_high_0_user.npy
-rw-r--r--   1 qin  staff   3.1M 17 May 05:57 interest_behaviors_high_1_user.npy
-rw-r--r--   1 qin  staff   3.1M 17 May 05:57 interest_behaviors_high_2_user.npy
-rw-r--r--   1 qin  staff   3.1M 17 May 05:57 interest_behaviors_high_3_user.npy
-rw-r--r--   1 qin  staff   3.1M 17 May 05:57 interest_behaviors_high_4_user.npy
-rw-r--r--   1 qin  staff   3.1M 17 May 05:57 interest_behaviors_low_0_user.npy
-rw-r--r--   1 qin  staff   3.1M 17 May 05:57 interest_behaviors_low_1_user.npy
-rw-r--r--   1 qin  staff   3.1M 17 May 05:57 interest_behaviors_low_2_user.npy
-rw-r--r--   1 qin  staff   3.1M 17 May 05:57 interest_behaviors_low_3_user.npy
-rw-r--r--   1 qin  staff   3

### convert user dict to user vecs

In [5]:
os.makedirs(infer_dir_processed, exist_ok=True)

In [6]:
# convert user dict to user vecs

for file_name in sorted(glob.glob(os.path.join(infer_dir, '*user.pkl'))):
    with open(file_name, 'rb') as f:
        user_dict = pickle.load(f)
        
    v_list = []
    for k, v in user_dict.items():
        v_list.append(v)

    v_vec = np.asarray(v_list)
    file_suffix = file_name.split("/")[-1].split(".pkl")[0]

    with open(os.path.join(infer_dir_processed, f'{file_suffix}.npy'), 'wb') as f:
        pickle.dump(v_vec, f)

In [7]:
!ls $infer_dir_processed

candiate_news_vec.npy              interest_behaviors_high_4_user.npy
candiate_nid_list.pkl              interest_behaviors_low_0_user.npy
interest_behaviors_high_0_user.npy interest_behaviors_low_1_user.npy
interest_behaviors_high_1_user.npy interest_behaviors_low_2_user.npy
interest_behaviors_high_2_user.npy interest_behaviors_low_3_user.npy
interest_behaviors_high_3_user.npy interest_behaviors_low_4_user.npy


### find vectors of candidate news

In [8]:
!wc -l $test_news_path

  120961 /Users/qin/2022/NewsRec/data/mind/large_test/news.tsv


In [9]:
nid_list = []
with open(test_news_path, 'r') as f:
    for line in f:
        line_split = line.split('\t')
        nid_list.append(line_split[0])
print(len(nid_list))

120961


In [10]:
# news_df = pd.read_csv(test_news_path, sep='\t', header=None)

In [11]:
# news_df.shape

In [12]:
with open(os.path.join(infer_dir, 'interest_behaviors_high_0_news.pkl'), 'rb') as f:
    full_news_dict = pickle.load(f)

In [13]:
len(nid_list), len(full_news_dict)

(120961, 120962)

In [14]:
nid2vecs = dict()
for idx, nid in enumerate(nid_list):
    nid2vecs[nid] = full_news_dict[idx+1]

In [15]:
news_df = pd.read_csv(candidate_news_file, sep='\t', header=None)
news_df.columns = ['nid', 'category', 'subcategory', 'title', 'abstract', 'url', 'entity1', 'entity2', 'timestamp']
news_df.shape

(22283, 9)

In [16]:
nid_list = news_df['nid'].tolist()
len(nid_list)

22283

In [17]:
nid_vecs_arr = []
candidate_nid = []
for nid in nid_list:
    nid_vecs_arr.append(nid2vecs[nid])
    candidate_nid.append(nid)

In [18]:
with open(os.path.join(infer_dir_processed, f'candiate_nid_list.pkl'), 'wb') as f:
     pickle.dump(candidate_nid, f)

In [19]:
nid_vecs_arr = np.asarray(nid_vecs_arr)

In [20]:
nid_vecs_arr.shape

(22283, 400)

In [21]:
with open(os.path.join(infer_dir_processed, f'candiate_news_vec.npy'), 'wb') as f:
     pickle.dump(nid_vecs_arr, f)

### results

In [22]:

# for file_name in sorted(glob.glob(os.path.join(infer_dir_processed, '*user.npy'))):
#     file_suffix = file_name.split("/")[-1].split(".npy")[0]
#     with open(os.path.join(infer_dir_processed, f'candiate_news_vec.npy'), 'rb') as f:
#          news_vec = pickle.load(f)
#     with open(file_name, 'rb') as f:
#          user_vec = pickle.load(f)
#     with open(os.path.join(infer_dir_processed, f'candiate_nid_list.pkl'), 'rb') as f:
#          candidate_nid = pickle.load(f)
            
#     bias_detection_df = pd.read_csv(bias_detection_path, header=None, sep='\t')
#     bias_detection_df.columns = ['nid', 'category', 'sub_category', 'bias_prob']

#     nis2bias_prob = dict(zip(bias_detection_df['nid'], bias_detection_df['bias_prob']))

#     a = np.dot(user_vec, news_vec.T)

#     top_n = 100
#     aa_arr = []
#     for aa in a:
#         ind = heapq.nlargest(top_n, range(len(aa)), aa.take)
#         aa_prob_list = [nis2bias_prob[candidate_nid[idx]] for idx in ind]
#         aa_arr.append(aa_prob_list)
#     aa_arr = np.asarray(aa_arr)

#     look_num_list = [i*10 for i in range(1, 11)]
#     mean_val_list = []
#     std_val_list = []
#     display_score_list = []
#     for look_num in look_num_list:
#         bb_arr = aa_arr[:, :look_num]
#         bias_prob_list = []
#         for bb in bb_arr:
#             temp = np.count_nonzero(bb >= 0.5)
#             bias_prob_list.append(temp/len(bb))
#         bias_prob_list = np.asarray(bias_prob_list)
        
#         mean_val = np.mean(bias_prob_list, dtype=np.float64)
#         std_val = np.std(bias_prob_list, dtype=np.float64)
#         display_score = f'{mean_val*100:.2f}'
#         mean_val_list.append(mean_val)
#         std_val_list.append(std_val)
#         display_score_list.append(display_score)
        
#     file_suffix = file_suffix.split("_user")[0]
#     print(file_suffix)
#     print('mean', mean_val_list)
#     print('std', std_val_list)
#     print('display', display_score_list)
#     print('*'*10)


interest_behaviors_high_0
mean [0.18645, 0.187125, 0.1831, 0.1821, 0.18067000000000003, 0.17935833333333334, 0.17797857142857143, 0.176525, 0.17482222222222218, 0.173815]
std [0.17283343860491812, 0.1551869336477785, 0.14510896671888412, 0.1403240535332414, 0.13622243244047583, 0.13378944667523918, 0.13130528204088646, 0.12902950001840666, 0.12649854534858748, 0.12488673178124247]
display ['18.64', '18.71', '18.31', '18.21', '18.07', '17.94', '17.80', '17.65', '17.48', '17.38']
**********
interest_behaviors_high_1
mean [0.1096, 0.10292500000000002, 0.10141666666666667, 0.1002375, 0.09962000000000001, 0.098575, 0.09759999999999999, 0.096575, 0.09568333333333334, 0.09524]
std [0.12384603344475752, 0.09657998951646247, 0.08500126633043639, 0.07842356848390668, 0.07360880110421579, 0.07091111366821612, 0.06808124798012014, 0.06595207066499126, 0.06413293269105627, 0.06253432977173419]
display ['10.96', '10.29', '10.14', '10.02', '9.96', '9.86', '9.76', '9.66', '9.57', '9.52']
**********
in